In [4]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from cupy import log as ln
import random
from cupy import random
import math
import scipy.stats as stats
import os
from matplotlib.pyplot import plot,savefig
import sys
filePath = 'Utils/cup'
sys.path.append(filePath)
import os
from functions import relu,drelu,softmax,divi_,turn_2_zero,scale,tanh,dtanh,sigmoid,dsigmoid
from optimizers import Adam,BN_layer
from cpsave import cp_save
from model_save import model_save

def padding(image, zero_num):
    if len(image.shape) == 4:
        image_padding = cp.zeros((image.shape[0],image.shape[1]+2*zero_num,image.shape[2]+2*zero_num,image.shape[3]))
        image_padding[:,zero_num:image.shape[1]+zero_num,zero_num:image.shape[2]+zero_num,:] = image
        return image_padding
    elif len(image.shape) == 3:
        image_padding = cp.zeros((image.shape[0]+2*zero_num, image.shape[1]+2*zero_num, image.shape[2]))
        image_padding[zero_num:image.shape[0]+zero_num, zero_num:image.shape[1]+zero_num,:] = image
        return image_padding
    else:
        print("维度错误")
        return 0
    


def pool(feature, size=2, stride=2):
    num,feature_h, feature_w, feature_ch = feature.shape
    pool_h = cp.uint16((feature_h - size)/stride + 1)
    pool_w = cp.uint16((feature_w - size)/stride + 1)
    feature_reshaped = feature.reshape(num,pool_h, feature_h//pool_h, pool_w, feature_w//pool_w, feature_ch)
    out = feature_reshaped.max(axis=2).max(axis=3)
    out_location_c = feature_reshaped.max(axis=2).argmax(axis=3)
    out_location_r = feature_reshaped.max(axis=4).argmax(axis=2)
    out_location = out_location_r * size + out_location_c
    return out, out_location

def rotate180(kernel, axis=(-2, -3)):
    return cp.flip(kernel, axis)
def add_bias(conv, bias):
    if conv.shape[-1] != bias.shape[0]:
        print("给卷积添加偏置维度出错")
    else:
        for i in range(bias.shape[0]):
            conv[:,:,:,i] += bias[i,0]
    return conv
def uni_permu(a1,b1,direction):
    a=a1*1
    b=b1*1
    if direction ==1:
        p = cp.random.permutation(len(a.T))
        return cp.array((a.T[p]).T), cp.array((b.T[p]).T)
    if direction == 0:
        p = cp.random.permutation(len(a))
        return cp.array((a[p])), cp.array((b[p]))
def mini_batch_generate(mini_batch_size,data1,label1):
    data = cp.array(data1*1)
    label =cp.array(label1*1)
    if (data.shape[0]%mini_batch_size == 0):
        n=data.shape[0]
    else:
        n = (int(data.shape[0]/mini_batch_size))*mini_batch_size
    data,label = uni_permu(data,label,0)
    mini_batches = cp.array([data[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)])
    mini_batches_labels =cp.array([label[k:k+mini_batch_size] for k in range(0,n,mini_batch_size)])
    mini_batches = mini_batches.reshape(int(data.shape[0]/mini_batch_size),mini_batch_size,data1.shape[1],data1.shape[2],data1.shape[3])
    mini_batches_labels = mini_batches_labels.reshape(int(label.shape[0]/mini_batch_size),mini_batch_size,label.shape[1],1)
    return (mini_batches),(mini_batches_labels)



def conv_cal_b(out_img_delta):
    #shape: 500, 10, 10, 16
    return cp.average(out_img_delta,axis = (0,1,2)).reshape(out_img_delta.shape[-1],1)

        
def pool_delta_error_bp(pool_out_delta1, pool_out_max_location1, size=2, stride=2):
    pool_out_delta = pool_out_delta1*1
    pool_out_max_location = pool_out_max_location1*1
    num,pool_h, pool_w, pool_ch = pool_out_delta.shape
    in_h = cp.uint16((pool_h-1)*stride+size)
    in_w = cp.uint16((pool_w-1)*stride+size)
    in_ch = pool_ch
    #复原尺寸
    pool_out_delta_reshaped = pool_out_delta.transpose(0,3,1,2)#16*5*5
    pool_out_delta_reshaped = pool_out_delta_reshaped.reshape(num*pool_h*pool_w*pool_ch)
    
    pool_out_max_location_reshaped = pool_out_max_location.transpose(0,3,1,2)
    pool_out_max_location_reshaped = pool_out_max_location_reshaped.reshape(num*pool_h*pool_w*pool_ch)
    
    in_delta_matrix = cp.zeros([num*pool_h*pool_w*pool_ch,size*size])
    
    in_delta_matrix[cp.arange(num*pool_h*pool_w*pool_ch), pool_out_max_location_reshaped] = pool_out_delta_reshaped
    
    in_delta = in_delta_matrix.reshape(num,pool_ch,pool_h, pool_w, size, size)
    in_delta = in_delta.transpose(0,2,4,3,5,1)
    in_delta = in_delta.reshape(num,in_h, in_w, in_ch)
    return  in_delta



def split_by_strides(x, kernel_size, stride=(1, 1)):
    """
    将张量按卷积核尺寸与步长进行分割
    :param x: 被卷积的张量
    :param kernel_size: 卷积核的长宽
    :param stride: 步长
    :return: y: 按卷积步骤展开后的矩阵
    """
    from cupy.lib.stride_tricks import as_strided
    B,C, h, w = x.shape
    out_H, out_W = (h - kernel_size[0]) // stride[0] + 1, (w - kernel_size[1]) // stride[1] + 1
    shape = (B,C, out_H, out_W, kernel_size[0], kernel_size[1])
    strides = (*x.strides[:-2], x.strides[-2] * stride[0],
               x.strides[-1] * stride[1], *x.strides[-2:])
    y = as_strided(x, shape, strides=strides)
    return y

def conv(img, conv_filter):
   
    if len(img.shape)!=4 or len(conv_filter.shape)!=4:
        print("卷积运算所输入的维度不符合要求")
        return 0
        
    if img.shape[-1] != conv_filter.shape[-1]:
        print("卷积输入图片与卷积核的通道数不一致")
        return 0
    img_2d = cp.copy(img)
    img_2d = img_2d.transpose(0,3,1,2)
    x_stride = split_by_strides(img_2d, conv_filter.shape[-3:-1], (1,1))*1
    filter2 = conv_filter.transpose(0,3,1,2)
    img_out = cp.tensordot(x_stride, filter2, axes=[(1, 4, 5), (1, 2, 3)]).transpose((0, 3, 1, 2))

    return img_out.transpose(0,2,3,1)




    return img_out
def reverse_conv2d(x1, kernel1, rotate=False, invert=False):
    """
    conv2d的反向卷积,求梯度时用的
    @param x: 被卷积的张量
    @param kernel: 卷积核
    @param rotate: 卷积核旋转180度
    @param invert: 该参数有点迷,不好解释,简单的说就是反向卷积有两种,视卷积结果的形状需要调整一些轴的位置
    @return: 反向卷积结果
    """
    kernel = kernel1.transpose(0,3,1,2)
    x = x1.transpose(0,3,1,2)
    ksize = kernel.shape
    x = split_by_strides(x, ksize[-2:])
    if rotate:
        kernel = rotate180(kernel)
    i = 0 if invert else 1
    y = cp.tensordot(x, kernel, [(i, 4, 5), (0, 2, 3)])
    if invert:
        return y.transpose((3, 1, 2,0))/x1.shape[0]
    return y.transpose((0, 3, 1, 2))
def split_by_strides2(X, kh, kw, s):
    N, H, W, C = X.shape
    oh = (H - kh) // s + 1
    ow = (W - kw) // s + 1
    shape = (N, oh, ow, kh, kw, C)
    strides = (X.strides[0], X.strides[1]*s, X.strides[2]*s, *X.strides[1:])
    A = cp.lib.stride_tricks.as_strided(X, shape=shape, strides=strides)
    return A
def conv2(img, conv_filter):
   
    if len(img.shape)!=4 or len(conv_filter.shape)!=4:
        print("卷积运算所输入的维度不符合要求")
        return 0
        
    if img.shape[-1] != conv_filter.shape[-1]:
        print("卷积输入图片与卷积核的通道数不一致")
        return 0
    img_num,img_h, img_w, img_ch = img.shape
    filter_num, filter_h, filter_w, img_ch = conv_filter.shape
    feature_h = img_h - filter_h + 1
    feature_w = img_w - filter_w + 1
    img_2d = cp.copy(img)   
    x_stride = split_by_strides2(img_2d, filter_h, filter_w, 1)*1
    filter2 = conv_filter.transpose(1,2,3,0)
    img_out = cp.tensordot(x_stride, filter2, axes=[(3,4,5), (0,1,2)]) 

    return img_out
import load_CIFAR10

In [5]:

def turn_2_zero(x1):
    import numpy
    x = cp.asnumpy(x1)
    y = numpy.int64(x>0)
    return cp.asarray(y)
## Data augmentation is necessary
def flip_ran(data1,p,dim):
    data = data1*1
    ran = turn_2_zero(p*cp.ones((data.shape))-cp.array([cp.random.uniform(0,1)*cp.ones((xx.shape)) for xx in data] ))
    data_b = ran*cp.flip(data*1,dim)+(1-ran)*data*1
    return data_b
def light_adjust(data1, a, b,p):
    data = data1*1
    temp = data*a+b
    temp2 = temp*1
    temp2[temp>1] = 1.0
    ran = turn_2_zero(p*cp.ones((data.shape))-cp.array([cp.random.uniform(0,1)*cp.ones((xx.shape)) for xx in data] ))
    data_b = ran*temp2+(1-ran)*data*1
    return data_b
def rotate(data1,p):
    data = data1*1
    temp2 = (cp.rot90(data, 1,axes=(1, 2)))
    ran = turn_2_zero(p*cp.ones((data.shape))-cp.array([cp.random.uniform(0,1)*cp.ones((xx.shape)) for xx in data] ))
    data_b = ran*temp2+(1-ran)*data*1
    return data_b
def Augmentation(data1,label1):
    num = 4
    data=cp.array(data1*1)
    label=cp.array(label1*1)
    lenth = data.shape[0]
    datab = cp.zeros((lenth*num,data.shape[1],data.shape[2],data.shape[3]))
    labelb = cp.zeros((lenth*num,label.shape[1],label.shape[2]))
    datab[0:lenth] = data*1
    datab[lenth:2*lenth] = flip_ran(data,0.7,2)
    datab[2*lenth:3*lenth] = rotate(datab[lenth:2*lenth],0.7)
    datab[3*lenth:4*lenth] = light_adjust(datab[2*lenth:3*lenth], cp.random.uniform(0.2,3.0), cp.random.uniform(0.0,1.5),0.7)
    labelb[0:lenth] = label*1
    labelb[1*lenth:2*lenth] = label*1
    labelb[2*lenth:3*lenth] = label*1
    labelb[3*lenth:4*lenth] = label*1
    del data1,label1,data,label
    return datab,labelb
train_data,train_label,test_data,test_label=load_CIFAR10.data_generate('cifar-10-batches-py')


In [7]:
class ConvNet(object):
    def __init__(self,k):
        channel = 3
        '''
        2层卷积，2层池化，3层全连接'''
        self.filters = [cp.random.normal(0,cp.sqrt(2/(3*3*channel)),(k, 3, 3, channel))] 
        self.filters_biases = [cp.zeros((k,1))]
        self.filters.append(cp.random.normal(0,cp.sqrt(2/(3*3*k)),(2*k, 3, 3, k)) )
        self.filters_biases.append(cp.zeros((2*k,1)))
        self.filters.append(cp.random.normal(0,cp.sqrt(2/(3*3*2*k)),(4*k, 3, 3, 2*k)) )
        self.filters_biases.append(cp.zeros((4*k,1)))
        self.filters.append(cp.random.normal(0,cp.sqrt(2/(3*3*4*k)),(8*k, 3, 3, 4*k)) )
        self.filters_biases.append(cp.zeros((8*k,1)))
        
        self.weights = [cp.random.normal(0,cp.sqrt(2/(8*k)),(10,8*k))]
        self.biases=[(cp.zeros((10,1)))]
        
        self.Adam_b= Adam(self.biases)
        self.Adam_w = Adam(self.weights)
        self.Adam_ft = Adam(self.filters)
        self.Adam_fb = Adam(self.filters_biases)
        self.BN_layer = [BN_layer(k*1,shape=2),BN_layer(2*k*1,shape=2),BN_layer(k*4,shape=2),BN_layer(k*8,shape=2)]


    def Backward(self, x,y,activate,dactivate,evaluate=False,train1=True):
        #第一层卷积
        conv_all=[]
        convy_all=[]
        pool_all=[x*1]
        pool_loc=[]
        out=[]
        pool_para = [1,2,2,8]              
        for i in range(4):   
            conv1 = add_bias(conv(padding(pool_all[i], 1), self.filters[i]),self.filters_biases[i])
            conv1_y = self.BN_layer[i].BN_backward(conv1,None,0,train=train1)
            relu1 = activate(conv1_y)
            pool1, pool1_max_locate = pool(relu1, size=pool_para[i], stride=pool_para[i])#num*32*32*k
            pool_all.append(pool1*1)
            conv_all.append(conv1*1)
            convy_all.append(conv1_y*1)
            pool_loc.append(pool1_max_locate*1)
            out.append(relu1*1)
        straight_input = pool_all[-1].reshape(pool_all[-1].shape[0],pool_all[-1].shape[1] * pool_all[-1].shape[2] * pool_all[-1].shape[3])
        active=[straight_input.T*1]#全连接层的输入数据
        active.append(softmax(cp.dot(self.weights[0],active[-1]))*1)#输出层的数据
        if evaluate == True:
            return active[-1]
        delta_full=[]
        delta_full.append((active[-1]-cp.squeeze(y.T))*1)#最后一层
        delta_full.append(cp.dot(self.weights[0].transpose(), delta_full[-1])*1)#第一层
        delta_convx=[]
        nabla_filters = [cp.zeros((b_s.shape)) for b_s in self.filters]
        nabla_bias = [cp.zeros((b_s.shape)) for b_s in self.filters_biases]
        
        
        for l in range(1, 5):
            if (l==1):
                delta_pool_ = (delta_full[-l].T).reshape(pool_all[-l].shape)
                delta_convy_ = pool_delta_error_bp(delta_pool_, pool_loc[-l],size=pool_para[-l],stride =pool_para[-l]) * dactivate(convy_all[-l])
                delta_convx_ = self.BN_layer[-l].BN_backward(conv_all[-l],delta_convy_,1,train=True)
                delta_convx.append(delta_convx_*1)
            else:
                delta_pool_ = conv(padding(delta_convx[-1], self.filters[-l+1].shape[1]-2), rotate180(self.filters[-l+1]).swapaxes(0,3))
                delta_convy_ = pool_delta_error_bp(delta_pool_, pool_loc[-l],size=pool_para[-l],stride =pool_para[-l]) * dactivate(convy_all[-l])
                delta_convx_ = self.BN_layer[-l].BN_backward(conv_all[-l],delta_convy_,1,train=True)
                delta_convx.append(delta_convx_*1)
            nabla_filters[-l] = reverse_conv2d(padding(pool_all[-l-1],1),delta_convx_,rotate=False,invert = True) 
            nabla_bias[-l] = (conv_cal_b(delta_convx_))
         
        
        nabla_w=[cp.zeros((b_s.shape)) for b_s in self.weights]
        nabla_b=[cp.zeros((b_s.shape)) for b_s in self.biases]
        for l in range(1):
            nabla_w[-l] = (cp.dot(delta_full[l-2], active[l-2].transpose())/(cp.shape(x)[0]))#按照顺序来的
            nabla_b[-l] = (cp.mean(delta_full[l-2],axis=1,keepdims=True))
 
        

        
        if evaluate == False:
            return nabla_w, nabla_b, nabla_filters, nabla_bias
        
    def evaluate(self, test_data1,test_label1,activate,dactivate):
        # 获得预测结果a:10*batch_size
        #testlabel:batch_size*10*1
        test_data = test_data1*1 
        test_label=test_label1*1
        data,label = mini_batch_generate(100,test_data,test_label)
        accuracy=[]
        cross = 0
        del test_data
        del test_label
        for i in range(data.shape[0]):
            a=self.Backward(data[i],label[i],activate,dactivate,evaluate=True,train1=False)
            max0=cp.argmax(a,axis=0).reshape(a.shape[1],1)
            max1=cp.argmax(label[i],axis=1)
            acc=((cp.sum((max0-max1) == 0))/(data[i].shape[0]))
            accuracy.append(acc)
            cross+=-cp.average(cp.sum((cp.squeeze(label[i]).T)*ln(a+pow(10,-30)),axis=0))
        return cp.average(accuracy),cross/data.shape[0]
        del data,label,a,max0,max1,acc
        

    def adam_update(self,lr,mini_batch_size,activate,dactivate,train_data_x,train_label_x):
        data_x=train_data_x*1
        label_x=train_label_x*1
        data1,label1 = mini_batch_generate(mini_batch_size,data_x,label_x)
        del data_x
        del label_x
        for j in range(data1.shape[0]):
#             data,label = Augmentation(data1[j]*1,label1[j]*1)
            delta_nabla_w, delta_nabla_b, delta_nabla_f, delta_nabla_fb =  self.Backward(data1[j],label1[j],activate,dactivate,evaluate=False)
            self.weights = self.Adam_w.New_theta(self.weights,delta_nabla_w,lr)
            self.biases =  self.Adam_b.New_theta(self.biases,delta_nabla_b,lr)
            self.filters = self.Adam_ft.New_theta(self.filters,delta_nabla_f,lr)
            self.filters_biases = self.Adam_fb.New_theta(self.filters_biases,delta_nabla_fb,lr)
            print('\r'+str(j)+'/'+str(int(data1.shape[0])),end='')
    def SGD(self,mini_batch_size,epoch,lr0,activate,dactivate):
        acc1_=[]
        for i in range(epoch):
            lr = divi_(lr0,i,40)
            self.adam_update(lr,mini_batch_size,activate,dactivate,train_data,train_label)
#             self.update_mini_batch(lr, mini_batch_size)
            print ("Epoch %s training complete" % i)
            acc1,cross = self.evaluate(test_data,test_label,activate,dactivate)
            print("the test Accuracy for task is:{} %".format((acc1)*100))
            print("the test loss for task is:{} ".format((cross)*100))
            acc1_.append(acc1*100)
        return acc1_
    

In [ ]:
net = ConvNet(64)
net.SGD(100,200,0.001,relu,drelu)
# import time
# time1 = time.time()
# net.evaluate(test_data,test_label,relu,drelu)
# time2 = time.time()
# print(time2-time1)
# x=net.Backward(train_data[0:10],train_label[0:10],relu,drelu,evaluate=False)
# acc=[]
# for i in range(1):
#     xx=net.SGD(100,200,0.001,relu,drelu)
#     acc.append(xx)

408/500